In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import random
import re

CHROME_DRIVER_PATH = "chromedriver.exe" 
CHROME_BINARY_LOCATION = r"C:\Program Files\Google\Chrome\Application\chrome.exe"

In [ ]:
# Hàm trích xuất dữ liệu chi tiết từ một khối HTML tin tuyển dụng TopCV (Giữ nguyên)
def extract_job_info_topcv(job_html_block):
    soup_job = BeautifulSoup(str(job_html_block), 'html.parser')
    data = {
        'Job_Name': "N/A",
        'Salary_Range': "N/A",
        'Company': "N/A",
        'Location': "N/A",
        'Required_Skills': "N/A",
        'Updated_Time': "N/A",
        'Detail_Link': "N/A",
    }
    
    try:
        # --- 1. Tiêu đề Công việc, Link và Lương ---
        title_block = soup_job.find('div', class_='title-block')
        if title_block:
            # Tên Công việc (Lấy từ data-original-title của thẻ <span>)
            title_span = title_block.find('span', {'data-original-title': True})
            if title_span:
                data['Job_Name'] = title_span.get('data-original-title', title_span.text.strip())
            
            # Link chi tiết (Lấy từ thẻ <a>)
            link_tag = title_block.find('a', target='_blank')
            if link_tag:
                data['Detail_Link'] = link_tag.get('href', 'N/A')
                
            # Mức lương
            salary_label = title_block.find('label', class_='title-salary')
            if salary_label:
                # Loại bỏ khoảng trắng và ký tự xuống dòng
                data['Salary_Range'] = salary_label.text.strip()
        
        # --- 2. Tên Công ty ---
        company_tag = soup_job.find('a', class_='company')
        if company_tag:
            # Lấy tên công ty sạch từ data-original-title
            data['Company'] = company_tag.get('data-original-title', company_tag.text.strip())
            
        # --- 3. Địa điểm ---
        address_label = soup_job.find('label', class_='address')
        if address_label:
            # Lấy địa điểm chi tiết từ data-original-title
            address_raw = address_label.get('data-original-title', address_label.text.strip())
            # Loại bỏ các tag HTML trong chuỗi
            data['Location'] = re.sub('<[^<]+?>', '', address_raw).strip()
            
        # --- 4. Tags Kỹ năng ---
        skills_div = soup_job.find('div', class_='skills')
        tags = []
        if skills_div:
            # Tìm tất cả các thẻ <label class="item">
            skill_items = skills_div.find_all('label', class_='item')
            for item in skill_items:
                # Loại trừ tag "3+" (nếu có, thường là tag đếm số lượng ẩn)
                tag_text = item.text.strip()
                if tag_text and not tag_text.endswith('+'):
                    tags.append(tag_text)
            data['Required_Skills'] = ", ".join(tags)

        # --- 5. Thời gian cập nhật ---
        deadline_label = soup_job.find('label', class_='deadline')
        if deadline_label:
            # Lấy text và làm sạch
            data['Updated_Time'] = deadline_label.text.strip().replace('Cập nhật', '').replace('Còn', '').replace('ngày để ứng tuyển', '').strip()

    except Exception as e:
        return None 

    return data

In [ ]:
# --- Thiết lập Cào Chính ---
def scrape_topcv_jobs(max_pages=10, keyword=""):
    
    # 1. Cấu hình WebDriver cho môi trường Local
    chrome_options = Options()
    chrome_options.binary_location = CHROME_BINARY_LOCATION
    
    # TÙY CHỌN AN TOÀN VÀ ANTI-BOT
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    chrome_options.add_argument('--no-sandbox') 
    chrome_options.add_argument('--log-level=3')

    # Bỏ qua các tùy chọn Profile và DevTools (để tránh lỗi)
    try:
        service = Service(CHROME_DRIVER_PATH)
        driver = webdriver.Chrome(service=service, options=chrome_options) 
        driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
            'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'
        })
    except Exception as e:
        print(f"LỖI KHỞI TẠO DRIVER. Vui lòng kiểm tra lại CHROME_DRIVER_PATH và CHROME_BINARY_LOCATION. Lỗi: {e}")
        return

    job_data = []
    base_url = "https://www.topcv.vn/viec-lam-it"
    search_keyword = keyword.replace(" ", "%20")

    # 2. Vòng lặp cào qua các trang
    for page in range(1, max_pages + 1):
        if search_keyword:
            # Cấu trúc search TopCV dùng tham số 'q' (query)
            url = f"{base_url}?q={search_keyword}&page={page}"
        else:
            url = f"{base_url}?page={page}"
            
        print(f"Đang cào trang: {page} | URL: {url}")
        
        try:
            driver.get(url)
            
            # *** PHẦN QUAN TRỌNG ĐÃ THAY ĐỔI: CUỘN TRANG ***
            scroll_count = 0
            while scroll_count < 3: # Cuộn 3 lần để đảm bảo tải hết nội dung động trên 1 trang
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(random.uniform(2, 4)) # Đợi nội dung tải
                scroll_count += 1
                
            # Đợi cuối cùng sau khi cuộn xong
            time.sleep(random.uniform(3, 5))


            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # 3. Tìm kiếm danh sách tin tuyển dụng
            # Selector chính: div có class job-item-2
            job_listings = soup.find_all('div', class_=lambda c: c and 'job-item-2' in c and 'job-item-default' in c)

            if not job_listings:
                print(f"Không tìm thấy tin tuyển dụng nào trên trang {page} hoặc đã hết trang (có thể bị chặn). Dừng cào.")
                break
            
            # 4. Trích xuất dữ liệu chi tiết
            jobs_scraped_on_page = 0
            for job_block in job_listings:
                data = extract_job_info_topcv(job_block)
                if data and data['Job_Name'] != 'N/A':
                    job_data.append(data)
                    jobs_scraped_on_page += 1
            
            if jobs_scraped_on_page == 0 and len(job_listings) > 0:
                 print(f"CẢNH BÁO: Đã tìm thấy {len(job_listings)} khối nhưng không trích xuất được data hợp lệ. Cấu trúc có thể đã bị thay đổi.")


            # Thêm độ trễ an toàn giữa các trang
            time.sleep(random.uniform(4, 7)) 
            
        except Exception as e:
            print(f"Lỗi không mong muốn trong quá trình cào trang {page}: {e}")
            break

    # 5. Đóng trình duyệt và lưu trữ
    driver.quit()
    
    if job_data:
        df = pd.DataFrame(job_data)
        file_name = f'topcv_job_trends_{search_keyword if search_keyword else "all"}_{len(df)}_tins.csv'
        df.to_csv(file_name, index=False, encoding='utf-8-sig')
        print(f"\nHoàn tất cào dữ liệu TopCV và lưu vào file '{file_name}'")
        print(f"Tổng số tin tuyển dụng đã cào: {len(df)}")
    else:
        print("\nKhông có dữ liệu nào được cào thành công.")

In [ ]:
if __name__ == "__main__":
    
    MAX_PAGES_TO_SCRAPE = 46
    KEYWORD = "" 
    
    scrape_topcv_jobs(max_pages=MAX_PAGES_TO_SCRAPE, keyword=KEYWORD)

Đang cào trang: 1 | URL: https://www.topcv.vn/viec-lam-it?page=1
Đang cào trang: 2 | URL: https://www.topcv.vn/viec-lam-it?page=2
Đang cào trang: 3 | URL: https://www.topcv.vn/viec-lam-it?page=3
Đang cào trang: 4 | URL: https://www.topcv.vn/viec-lam-it?page=4
Đang cào trang: 5 | URL: https://www.topcv.vn/viec-lam-it?page=5
Đang cào trang: 6 | URL: https://www.topcv.vn/viec-lam-it?page=6
Đang cào trang: 7 | URL: https://www.topcv.vn/viec-lam-it?page=7
Đang cào trang: 8 | URL: https://www.topcv.vn/viec-lam-it?page=8
Đang cào trang: 9 | URL: https://www.topcv.vn/viec-lam-it?page=9
Đang cào trang: 10 | URL: https://www.topcv.vn/viec-lam-it?page=10
Đang cào trang: 11 | URL: https://www.topcv.vn/viec-lam-it?page=11
Đang cào trang: 12 | URL: https://www.topcv.vn/viec-lam-it?page=12
Đang cào trang: 13 | URL: https://www.topcv.vn/viec-lam-it?page=13
Đang cào trang: 14 | URL: https://www.topcv.vn/viec-lam-it?page=14
Đang cào trang: 15 | URL: https://www.topcv.vn/viec-lam-it?page=15
Đang cào tran